In [ ]:
import numpy as np
from scipy.cluster.vq import vq 

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Circle, Arc, Polygon, FancyBboxPatch, RegularPolygon
from matplotlib.colors import _to_rgba_no_colorcycle

from cellcomplex.property_topomesh.example_topomesh import hexagon_topomesh, hexagonal_grid_topomesh
from cellcomplex.property_topomesh.visualization.matplotlib import mpl_draw_topomesh
from cellcomplex.property_topomesh.analysis import compute_topomesh_property
from cellcomplex.property_topomesh.extraction import sub_topomesh
            
mpl.rcParams['font.family'] = "Avenir Next"
mpl.rcParams['font.weight'] = 'bold'

In [ ]:
n_rows = 1
n_cols = int(np.floor(n_rows*np.sqrt(2)))

# card_width = 29.7/n_cols
card_width = card_height = np.minimum(21/n_rows, 29.7/n_cols)
card_height = 21/n_rows

pz_size = 2

cell_size = (0.8*card_width/(2*(pz_size+1)+1))/2
# cell_size = 0.25

board_topomesh = hexagonal_grid_topomesh(size=pz_size+1, cell_size=cell_size)

compute_topomesh_property(board_topomesh, 'barycenter', 2)

cells = np.array(list(board_topomesh.wisps(2)))
cell_centers = board_topomesh.wisp_property('barycenter', 2).values(cells)

cell_distances = np.linalg.norm(cell_centers, axis=-1)

cz_cells = cells[cell_distances < 2.5*cell_size]
pz_cells = cells[cell_distances >= 2.5*cell_size]

p0_cells = cells[cell_centers[:, 0] >= 3.5*cell_size]


In [ ]:
figure = plt.figure(figsize=(29.7, 21))
ax = figure.gca()

for card_row in range(n_rows):
    for card_column in range(n_cols):
        card_x = -29.7/2 + (card_column + 0.5)*card_width
        card_y = -21/2 + (card_row + 0.5)*card_height

        card = FancyBboxPatch(
            xy=(card_x-0.8*card_width/2, card_y-0.8*card_height/2), width=0.8*card_width, height=0.8*card_height, 
            boxstyle="Round", fc='none', ec='k', linewidth=3)
        ax.add_artist(card)
        
        board_center = np.array([card_x, card_y, 0])        
        # board_center = np.array([card_x, card_y + 0.8*card_height/4, 0])
        
        radius = cell_size*(np.sqrt(3)*(pz_size+2))
        c = Circle(xy=board_center, radius=radius, fc='#99cc66', ec='#99cc66', alpha=0.2, linewidth=1)
        ax.add_artist(c)
        
        cz_topomesh = sub_topomesh(board_topomesh, 2, cz_cells)
        cz_topomesh.update_wisp_property('barycenter', 0, {v: p + board_center for v, p in cz_topomesh.wisp_property('barycenter', 0).items()})
        pz_topomesh = sub_topomesh(board_topomesh, 2, pz_cells)
        pz_topomesh.update_wisp_property('barycenter', 0, {v: p + board_center for v, p in pz_topomesh.wisp_property('barycenter', 0).items()})
        p0_topomesh = sub_topomesh(board_topomesh, 2, p0_cells)
        p0_topomesh.update_wisp_property('barycenter', 0, {v: p + board_center for v, p in p0_topomesh.wisp_property('barycenter', 0).items()})
        compute_topomesh_property(p0_topomesh, 'barycenter', 1)
        compute_topomesh_property(p0_topomesh, 'barycenter', 2)
        
        mpl_draw_topomesh(pz_topomesh, figure, 1, color='forestgreen', linewidth=1)
        mpl_draw_topomesh(pz_topomesh, figure, 2, color='forestgreen', alpha=0.2)
        
        mpl_draw_topomesh(cz_topomesh, figure, 1, color='darkmagenta', linewidth=1)
        mpl_draw_topomesh(cz_topomesh, figure, 2, color='darkmagenta', alpha=0.2)
        
        # mpl_draw_topomesh(p0_topomesh, figure, 1, color='darkturquoise', linewidth=1, coef=0.5)
        mpl_draw_topomesh(p0_topomesh, figure, 2, color='dodgerblue', alpha=0.8, coef=0.5)

        target_cell = pz_cells[np.random.randint(len(pz_cells))]
        x = board_topomesh.wisp_property('barycenter', 2)[target_cell][0]
        while np.random.normal(loc=1, scale=0.1) < 1 - (x/cell_size + 5)/10:
            target_cell = pz_cells[np.random.randint(len(pz_cells))]
            x = board_topomesh.wisp_property('barycenter', 2)[target_cell][0]
        print((x/cell_size + 5)/10)
        
        neighbor_cells = list(pz_topomesh.border_neighbors(2, target_cell))

        # n_neighbors = 3
        n_neighbors = int(np.round(np.random.normal(loc=4, scale=1)))
        
        # neighbor_targets = list(np.random.permutation(neighbor_cells)[:n_neighbors])
        # neighbor_targets = list(np.random.permutation(neighbor_cells)[:2])
        neighbor_targets = []
        while len(neighbor_targets) < n_neighbors:
            if len(neighbor_targets) > 0:
                neighbor_neighbor_cells = np.concatenate([list(pz_topomesh.border_neighbors(2, n)) for n in neighbor_targets])
            else:
                neighbor_neighbor_cells = []
            neighbor_neighbor_cells = set(neighbor_cells).union(neighbor_neighbor_cells).difference(neighbor_targets).difference({target_cell})
            neighbor_targets += [np.random.permutation(list(neighbor_neighbor_cells))[0]]
       
        target_topomesh = sub_topomesh(pz_topomesh, 2, [target_cell])
        mpl_draw_topomesh(target_topomesh, figure, 2, color='r', alpha=0.8)
        mpl_draw_topomesh(target_topomesh, figure, 1, color='r', linewidth=1)

        if len(neighbor_targets) > 0:
            target_neighbors_topomesh = sub_topomesh(pz_topomesh, 2, neighbor_targets)
            mpl_draw_topomesh(target_neighbors_topomesh, figure, 2, color='r', alpha=0.2)
            mpl_draw_topomesh(target_neighbors_topomesh, figure, 1, color='r', linewidth=1)

        target_center = board_topomesh.wisp_property('barycenter', 2).values([target_cell]+neighbor_targets).mean(axis=0)
        target_angle = np.degrees(np.arctan2(target_center[1], target_center[0]))

        hex_radius = 1.0*radius
        
        h = RegularPolygon(xy=board_center[:2], radius=hex_radius, numVertices=6, orientation=np.radians(30), fc='none', ec='grey', alpha=0.25, linewidth=1)
        ax.add_artist(h)
        
        h = RegularPolygon(xy=board_center[:2], radius=1.05*hex_radius, numVertices=6, orientation=np.radians(30), fc='none', ec='grey', alpha=0.33, linewidth=2)
        ax.add_artist(h)
        
        s = np.round((target_angle - 30)/60)
        t_min = np.radians(s*60 + 0)
        t_max = np.radians((s+1)*60 - 0)
        ax.plot(
            board_center[0] + hex_radius*(0.75*np.cos([t_min, t_max]) + 0.25*np.cos([t_max, t_min])) ,
            board_center[1] + hex_radius*(0.75*np.sin([t_min, t_max]) + 0.25*np.sin([t_max, t_min])) ,
            color='r', alpha=0.8, linewidth=5
        )

        triangle_points = board_center + hex_radius*np.array([
            # [0.25*np.cos(o) + 0.75*np.cos(0), 0.25*np.sin(o) + 0.75*np.sin(0), 0]
            [0.15*np.cos(o) + 0.85*np.cos(0), 0.15*np.sin(o) + 0.85*np.sin(0), 0]
            for o in np.radians([-60, 0, 60])
        ])
        t = Polygon(triangle_points[:, :2], color='grey', ec='#333333', alpha=0.33, zorder=10)
        # ax.add_artist(t)


ax.set_xlim(-14.85, 14.85)
ax.set_ylim(-10.5, 10.5)
ax.axis('off')
